# 04 - Applied ML

## Deadline
Monday November 21, 2016 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution
you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code

## Background
In this homework we will gain experience on Applied Machine Learning, exploring an interesting dataset about soccer players and referees.
You can find all the data in the `CrowdstormingDataJuly1st.csv` file, while you can read a thorough [dataset description here](DATA.md).
Given that the focus of this homework is Machine Learning, I recommend you to first take a look at [this notebook](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb)
containing a solid work in pre-processing + visualization of the given dataset. You are *not* allowed to just copy/paste the pre-processing steps
performed by the notebook authors -- you are still supposed to perform your own data analysis for the homework. Still, I'm confident that consulting first
the work done by expert data analysts will speed up tangibly your effort (i.e., they have already found for you many glitches in the data :)


## Assignment
1. Train a `sklearn.ensemble.RandomForestClassifier` that given a soccer player description outputs his skin color. Show how different parameters 
passed to the Classifier affect the overfitting issue. Perform cross-validation to mitigate the overfitting of your model. Once you assessed your model,
inspect the `feature_importances_` attribute and discuss the obtained results. With different assumptions on the data (e.g., dropping certain features even
before feeding them to the classifier), can you obtain a substantially different `feature_importances_` attribute?
*BONUS*: plot the learning curves against at least 2 different sets of parameters passed to your Random Forest. To obtain smooth curves, partition
your data in at least 20 folds. Can you find a set of parameters that leads to high bias, and one which does not?

2. Aggregate the referee information grouping by soccer player, and use an unsupervised learning technique to cluster the soccer players in 2 disjoint
clusters. Remove features iteratively, and at each step perform again the clustering and compute the silhouette score -- can you find a configuration of features with high silhouette
score where players with dark and light skin colors belong to different clusters? Discuss the obtained results.


--------------------------------------------------------------------------------------------------------------
# Data Description

From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3147) that these players played under in their professional career. We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.
 
Player photos were available from the source for 1586 out of 2053 players. Players’ skin tone was coded by two independent raters blind to the research question who, based on their profile photo, categorized players on a 5-point scale ranging from “very light skin” to “very dark skin” with “neither dark nor light skin” as the center value. 

Additionally, implicit bias scores for each referee country were calculated using a race implicit association test (IAT), with higher values corresponding to faster white | good, black | bad associations. Explicit bias scores for each referee country were calculated using a racial thermometer task, with higher values corresponding to greater feelings of warmth toward whites versus blacks. Both these measures were created by aggregating data from many online users in referee countries taking these tests on [Project Implicit](http://projectimplicit.net).

In all, the dataset has a total of 146028 dyads of players and referees. A detailed description of all variables in the dataset can be seen in the list below.

## Variables:

*playerShort* - short player ID

*player* - player name

*club* - player club

*leagueCountry* - country of player club (England, Germany, France, and Spain)

*birthday* - player birthday

*height* - player height (in cm)

*weight* - player weight (in kg)

*position* - detailed player position

*games* - number of games in the player-referee dyad

*victories* - victories in the player-referee dyad

*ties* - ties in the player-referee dyad

*defeats* - losses in the player-referee dyad

*goals* - goals scored by a player in the player-referee dyad

*yellowCards* - number of yellow cards player received from referee

*yellowReds* - number of yellow-red cards player received from referee

*redCards* - number of red cards player received from referee

*photoID* - ID of player photo (if available)

*rater1* - skin rating of photo by rater 1 (5-point scale ranging from “very light skin” to “very dark skin”)

*rater2* - skin rating of photo by rater 2 (5-point scale ranging from “very light skin” to “very dark skin”)

*refNum* - unique referee ID number (referee name removed for anonymizing purposes)

*refCountry* - unique referee country ID number (country name removed for anonymizing purposes)

*meanIAT* - mean implicit bias score (using the race IAT) for referee country, higher values correspond to faster white | good, black | bad associations

*nIAT* - sample size for race IAT in that particular country

*seIAT* - standard error for mean estimate of race IAT

*meanExp* - mean explicit bias score (using a racial thermometer task) for referee country, higher values correspond to greater feelings of warmth toward whites versus blacks

*nExp* - sample size for explicit bias in that particular country

*seExp* - standard error for mean estimate of explicit bias measure




--------------------------------------------------------------------------------------------------------------

# IMPLEMENTATION
## Supervised Classification

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score
import preprocessing_helper as pprh
import classifiers_helper as clfh
import aggregater as aggr
from statistics import mode

In [2]:
data = 'CrowdstormingDataJuly1st.csv'
dyads = pd.read_csv(data)
nb_cols = len(dyads.columns)
print(nb_cols)

28


In [3]:
dyads.ix[:10, :14]

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals,yellowCards
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,1,0,0
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,1,0,1
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,0,0,1
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,0,0,0
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,0,0,0
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,1,0,0
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,0,0,0,0
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,0,1,0,0
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,1,0,0,0
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,0,0,0,0


In [4]:
dyads.ix[:10, 14:28]

,yellowReds,redCards,photoID,rater1,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0,0,95212.jpg,0.25,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,0,0,1663.jpg,0.75,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,0,0,NaN,NaN,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
5,0,0,3868.jpg,0.25,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0,0,47704.jpg,0.00,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0,0,22356.jpg,1.00,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
8,0,0,16528.jpg,0.25,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
9,0,0,36499.jpg,0.00,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


Through the visualization of the data and the ipython notebook which previously worked on the same dataset, we observed that each data entry is a dyad between a player and a referee. They all contain a player ID, a referee ID, their respective descriptors and some informations about the dyad itself.

We could define a dyad as a relationship between a player and a referee. A dyad can contain multiple encounters of the pair (player <-> referee) from different matches. 



### Cleaning (1)

First of all, as we have (player,referee) dyads, a referee in our dataset should have arbitrated at least one match. This implies that we should find at least 22 dyads containing the given referee (a team has 11 players).
We thus check that for each referee we obtain at least 22 dyads:

In [5]:
referees = pd.DataFrame(dyads.groupby(['refNum']).size())
referees.columns = ['count']
bad_refs = referees[referees['count'] < 22]
nb_refs = len(referees)
nb_badrefs = len(bad_refs)
print("Number of refs: "+ str(nb_refs))
print("Number of refs with missing data: "+str(nb_badrefs))
print("Quantity of refs with missing data: " + str(100*(nb_badrefs/nb_refs)) + "%")

Number of refs: 3147
Number of refs with missing data: 2149
Quantity of refs with missing data: 68.28725770575151%


We found 2149 referees contained in less than 22 dyads. We can consider these as missing data events. 

In [6]:
to_rm = bad_refs.index

dyads_w_rm = dyads.drop(to_rm)
nb_dyads = len(dyads)
nb_dyads_wrm = len(dyads_w_rm)
print("Number of dyads: "+ str(len(dyads)))
print("Number of dyads after deletions: "+str(nb_dyads_wrm))
print("Percent of dyads lost: "+str(100*(1-(nb_dyads_wrm/nb_dyads)))+"%")

Number of dyads: 146028
Number of dyads after deletions: 143879
Percent of dyads lost: 1.4716355767387057%


We see that even that the referees to delete represented 68% of the set of referees, the percentage of lost dyads after a potential removal is only around 1.5%. We thus decide to apply the deletion.

In [7]:
dyads = dyads.drop(to_rm)

### Setting of model's Target

Our goal is to define if a player has dark skin or light skin. We thus need to design a model and train it on a part of this data with predefined skin labels. We will take the columns 'rater1' and 'rater2' to define this label. As the model needs an output (a valid skin label) we need at least one of these columns to be valid.

In [8]:
dyads = dyads[~(np.isnan(dyads['rater1']) & np.isnan(dyads['rater2']))]
dyads.reset_index(inplace=True)

As the label is defined by two columns we want these columns to agree with each other.

In [9]:
max_diff = (dyads['rater1']-dyads['rater2']).max()
max_diff

0.25

In [10]:
dyads['rater1'].unique()

array([ 0.25,  1.  ,  0.5 ,  0.  ,  0.75])

The scale is going from 0 to 1 with a step of 0.25 and the max difference is 0.25.
Therefore we cannot find a case where there are opposite opinions about a player (one black, the other white)

We choose to transform this scale of rating into binary labels. We consider our model to predict more of a white/coloured ouput. 

First approach would be to take the mean of these values and if the result is smaller than 0.5 we want to output 0 ('white') and if it is greater than 0.5 output 1 ('black/coloured').

Beforehand, it is important to check if both values are equal to 0.5. Indeed, this method won't be able to set a label to the given entry. We thus check if our dataset contains such values:

In [11]:
dyads[(dyads['rater1'] == 0.5) & (dyads['rater2'] == 0.5)]['playerShort'].count()

8837

We find 8989 cases where both raters find 0.5 as rating. We decide that those players will be represented by a 1 ('black').

In [12]:
dyads['y'] = (dyads['rater1'] + dyads['rater2']) / 2 

In [13]:
dyads.loc[dyads['y'] < 0.5, 'y'] = 0
dyads.loc[dyads['y'] >= 0.5, 'y'] = 1

Since we want a model that classifies players based on their descriptions, we first need to transform our data into a frame with an entry per player. Therefore, this implies, aggregating the dyads players while applying different aggregation functions per columns based on the information it contains. We will thus need to make some choices of interpretation to do the latter.

### Aggregating and cleaning (2)
We first aggregate the dyads by players while appending their column informations into arrays. We also need  to not reorder the arrays to keep a mapping between a referee and its informations concerning the player.

In [14]:
players = dyads.groupby(['playerShort'])
players = pprh.groups_to_lists(players, 'playerShort')

In [15]:
naive_data = pprh.drop_columns(players,['birthday', 'player', 'photoID', 'Alpha_3', 'refCountry', 'nIAT', 'nExp','index', 'refNum', 'rater1', 'rater2'])

We have a repetition of values for some columns that are supposed to be constant: 
weight, height, club, y (skin rate). We check if the respective lists of those columns per player contain the same value or not.

In [16]:
print(pprh.has_same_value(naive_data['height']))
print(pprh.has_same_value(naive_data['weight']))
print(pprh.has_same_value(naive_data['y']))
print(pprh.has_same_value(naive_data['club']))
print(pprh.has_same_value(naive_data['leagueCountry']))

False
False
True
True
True


As we can see, height and weight can differ for a given player. Is it because the list contains NaN or just different values?

In [17]:
print(pprh.has_nan(naive_data['height']))
print(pprh.has_nan(naive_data['weight']))

True
True


There are NaNs in the height and weight columns for some players. We will replace those by their correct height and weight if we have them in the data.

In [18]:
naive_data['height'] = pprh.replace_nan(naive_data['height'])
naive_data['weight'] = pprh.replace_nan(naive_data['weight'])

print(pprh.has_nan(naive_data['height']))
print(pprh.has_nan(naive_data['weight']))

False
False


So let's check if the value is the same in the list for each player.

In [19]:
print(pprh.has_same_value(naive_data['height']))
print(pprh.has_same_value(naive_data['weight']))

False
False


Let's see if one of our player has no height or weight

In [20]:
unique_height = pd.DataFrame(naive_data['height'].apply(lambda row: len(set(row))))
player_nan_height = unique_height[unique_height['height']==0]
unique_weight = pd.DataFrame(naive_data['weight'].apply(lambda row: len(set(row))))
player_nan_weight = unique_weight[unique_weight['weight']==0]

Drop them !!!!

In [21]:
list_diff = set(list(player_nan_weight.index.values) + list(player_nan_height.index.values))

In [22]:
naive_data = naive_data.drop(list_diff)

Looks like no... So for each player let's take the mean of its height and weight.

In [23]:
print(pprh.has_same_value(naive_data['height']))
print(pprh.has_same_value(naive_data['weight']))

True
True


### Reducing
We apply different aggregation functions on our columns:
- As discussed, the columns  club, leagueCountry, height and weight are constants. We only take the first element of the array. This is also the case for y since a player do not change skin color over time.
- Since the player's position may change over time, we decide to apply a majority vote on the list of his positions. We also create a "Unknown" position class for the Na cases.
- The columns games, victories, ties, defeats, goals, yellowCards, redCards and yellowReds are simply summed.
- seIAT and seExp becomes the standard deviation of the meanIAT and meanExp a player has.
- We take the average of meanIAT and meanExp

In [24]:
naive_data['y'] = naive_data['y'].apply(lambda x: int(x[0]))
naive_data['club'] = naive_data['club'].apply(lambda x: x[0])
naive_data['leagueCountry'] = naive_data['leagueCountry'].apply(lambda x: x[0])
naive_data['height'] = naive_data['height'].apply(lambda x: x[0])
naive_data['weight'] = naive_data['weight'].apply(lambda x: x[0])
#majority vote
naive_data['position'] = naive_data['position'].apply(mode)
naive_data['position'].fillna("Unknown", inplace=True)
naive_data['games'] = naive_data['games'].apply(lambda x: sum(x))
naive_data['victories'] = naive_data['victories'].apply(lambda x: sum(x))
naive_data['ties'] = naive_data['ties'].apply(lambda x: sum(x))
naive_data['defeats'] = naive_data['defeats'].apply(lambda x: sum(x))
naive_data['goals'] = naive_data['goals'].apply(lambda x: sum(x))
naive_data['yellowCards'] = naive_data['yellowCards'].apply(lambda x: sum(x))
naive_data['yellowReds'] = naive_data['yellowReds'].apply(lambda x: sum(x))
naive_data['redCards'] = naive_data['redCards'].apply(lambda x: sum(x))
naive_data['seIAT'] = naive_data['meanIAT'].apply(np.std)
naive_data['seExp'] = naive_data['meanExp'].apply(np.std)
naive_data['meanIAT'] = naive_data['meanIAT'].apply(np.mean)
naive_data['meanExp'] = naive_data['meanExp'].apply(np.mean)
naive_data.head()

,club,defeats,games,goals,height,leagueCountry,meanExp,meanIAT,position,redCards,seExp,seIAT,ties,victories,weight,y,yellowCards,yellowReds
playerShort,,,,,,,,,,,,,,,,,,
aaron-hughes,Fulham FC,222,627,9,182.0,England,0.495149,0.346709,Center Back,0,0.231539,0.031525,171,234,71.0,0,19,0
aaron-hunt,Werder Bremen,120,334,62,183.0,Germany,0.448310,0.349060,Attacking Midfielder,1,0.173776,0.022303,73,141,73.0,0,42,0
aaron-lennon,Tottenham Hotspur,112,399,30,165.0,England,0.491392,0.346055,Right Midfielder,0,0.225670,0.028481,96,191,63.0,0,10,0
aaron-ramsey,Arsenal FC,65,251,37,178.0,England,0.515506,0.346706,Center Midfielder,1,0.250735,0.032627,42,144,76.0,0,31,0
abdelhamid-el-kaoutari,Montpellier HSC,43,124,1,180.0,France,0.335587,0.331600,Center Back,2,0.238918,0.045100,40,41,73.0,0,8,4


Before training our classifier, we drop our nan in height and weight.

In [25]:
naive_data.dropna(inplace=True)

In [26]:
labels = naive_data.pop('y')

We encode our categorical data columns. The columns club, leagueCountry and position are supposed to be categorical data. We could dummily encode them with indices as scikit's label_encoding does. However, sklearn's RandomForestClassifier consider those values as continuous. At a given branch, it will separate the obtained values through intervals making the distance between indices important.
Therefore we decide to one-hot encode our data. It slightly increases our feature dimensions but will reduce our bias.

In [27]:
to_encode = ['club', 'leagueCountry','position']
encoded_data = pprh.one_hot_encode(naive_data, to_encode)

Random Forest Classifier: check this out!

In [28]:
#CV then score (=prediction)
X_train, X_test, y_train, y_test = train_test_split(encoded_data, labels, test_size=0.20, random_state=0)
rfclf = RandomForestClassifier(n_estimators=10).fit(X_train, y_train) 
score = rfclf.score(X_test, y_test)
print("Classic score")
print(score)

#Classifier then CV K-Fold (k=3, default)
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, encoded_data, labels, cv=20)
print("Cross-val score")
print(scores.mean())

Classic score
0.778156996587
Cross-val score
0.791485920852


So we got a score around 78% but we know that we are losing informations on the cards given by different referees. It means that for a player the value of a card given by a "racist" referee is the same as the value of a card given by a "not racist" referee. We don't want to lose this information, so we will need to ponder the cards using the "racist" value given for a referee.

First of all we normalize the value of meanIAT and meanEXP in our dyads dataset.

In [29]:
norm_dyads = pd.DataFrame(dyads)
norm_dyads['meanIATNorm'] = (norm_dyads['meanIAT'] - norm_dyads['meanIAT'].min()) / (norm_dyads['meanIAT'].max() - norm_dyads['meanIAT'].min())
norm_dyads['meanExpNorm'] = (norm_dyads['meanExp'] - norm_dyads['meanExp'].min()) / (norm_dyads['meanExp'].max() - norm_dyads['meanExp'].min())

In [30]:
norm_dyads['refScore']=(norm_dyads['meanIAT']+norm_dyads['meanExp'])/2
norm_dyads['yellowCardsPonder']=norm_dyads['refScore'] * norm_dyads['yellowCards']
norm_dyads['redCardsPonder']=norm_dyads['refScore'] * norm_dyads['redCards']
norm_dyads['yellowRedsPonder']=norm_dyads['refScore'] * norm_dyads['yellowReds']
norm_dyads.drop(['yellowCards', 'redCards', 'yellowReds'], axis=1, inplace=True)

We do the same changes as above on the new dyad data.TODO: reformulate

In [31]:
players_norm = aggr.from_dyads_to_players_aggregate(norm_dyads)

In [32]:
players_norm.drop(['meanIATNorm', 'meanExpNorm'], axis=1, inplace=True)

In [33]:
labels_norm = players_norm.pop('y')

In [34]:
to_encode_norm = ['club', 'leagueCountry','position']
#encoded_data_norm = pprh.label_encode(players_norm, to_encode_norm)
encoded_data_norm = pprh.one_hot_encode(players_norm, to_encode_norm)

In [35]:
#Classifier then CV K-Fold (k=3, default)
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, encoded_data_norm, labels_norm, cv=8)
print("Cross-val score")
print(scores.mean())

Cross-val score
0.79070804092


### Feature Importances Analysis

We now have satisfying data preprocessing and classification. We will look at the influence of the features on our model. In other words, we will measure the feature importances as well as the model behaviour in the case of feature modifications (mainly deletion).

In [39]:
#Feature Importances
(values, labels) = clfh.plot_importances(clf, encoded_data_norm, labels_norm)

We remark that the features that often comes at the top of our plot are the two standard deviation of IAT and Exp and also their mean value. Let's now tweak our data by removing/encoding/adding some columns and see if we can get different important features

We will first try to not encode the column using Hot Encoder but just using Label Encoder.

In [40]:
label_encoded_data_norm = pprh.label_encode(players_norm, to_encode_norm)

In [43]:
#Classifier then CV K-Fold (k=3, default)
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, label_encoded_data_norm, labels_norm, cv=8)
print("Cross-val score")
print(scores.mean())

Cross-val score
0.788684968957


In [44]:
#Feature Importances
(values, labels) = clfh.plot_importances(clf, label_encoded_data_norm, labels_norm)

## Bonus

## Unsupervised Classification